In [1]:
from pyspark.sql import SparkSession, functions as fn

In [2]:
spark = SparkSession.builder.getOrCreate()

22/05/05 14:41:47 WARN Utils: Your hostname, homebook.local resolves to a loopback address: 127.0.0.1; using 192.168.1.104 instead (on interface en7)
22/05/05 14:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 14:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = spark.read.parquet("../data/ingested/ingest_with_input_manager/")

In [5]:
df.where(fn.col("total_revenue") < 0).show()

+-------+----------+------+------------------+-------------+--------+-----------------+-----+-------------------+-----------------------+----------+-----------------+-------------+--------------------+----------------------+---------------------+----------+
|site_id|ad_type_id|geo_id|device_category_id|advertiser_id|order_id|line_item_type_id|os_id|integration_type_id|monetization_channel_id|ad_unit_id|total_impressions|total_revenue|viewable_impressions|measurable_impressions|revenue_share_percent|      date|
+-------+----------+------+------------------+-------------+--------+-----------------+-----+-------------------+-----------------------+----------+-----------------+-------------+--------------------+----------------------+---------------------+----------+
|    351|        10|   113|                 2|           79|    3487|                8|   58|                  1|                     19|      5179|               22|      -0.1542|                   8|                    22|  

In [6]:
df = df.where(fn.col("total_revenue") >= 0)

In [7]:
df.where(fn.col("total_revenue") < 0).show()

+-------+----------+------+------------------+-------------+--------+-----------------+-----+-------------------+-----------------------+----------+-----------------+-------------+--------------------+----------------------+---------------------+----+
|site_id|ad_type_id|geo_id|device_category_id|advertiser_id|order_id|line_item_type_id|os_id|integration_type_id|monetization_channel_id|ad_unit_id|total_impressions|total_revenue|viewable_impressions|measurable_impressions|revenue_share_percent|date|
+-------+----------+------+------------------+-------------+--------+-----------------+-----+-------------------+-----------------------+----------+-----------------+-------------+--------------------+----------------------+---------------------+----+
+-------+----------+------+------------------+-------------+--------+-----------------+-----+-------------------+-----------------------+----------+-----------------+-------------+--------------------+----------------------+--------------------

In [13]:
df.groupBy("site_id", "date").count().show()

+-------+----------+-----+
|site_id|      date|count|
+-------+----------+-----+
|    351|2019-06-20| 3752|
|    342|2019-06-20| 1111|
|    344|2019-06-23| 1146|
|    346|2019-06-23| 4689|
|    349|2019-06-23| 1504|
|    342|2019-06-17| 1087|
|    351|2019-06-17| 3587|
|    349|2019-06-17| 1653|
|    350|2019-06-17| 3027|
|    347|2019-06-20| 1132|
|    348|2019-06-23|  462|
|    349|2019-06-20| 1487|
|    350|2019-06-23| 3093|
|    345|2019-06-17| 2422|
|    345|2019-06-20| 2538|
|    350|2019-06-20| 3518|
|    351|2019-06-23| 3403|
|    348|2019-06-20|  589|
|    343|2019-06-23| 2070|
|    342|2019-06-23|  987|
+-------+----------+-----+
only showing top 20 rows



In [8]:
df = df.withColumn("unpaid_total_impressions", fn.when(fn.col("total_revenue") > 0.0001, 0).otherwise(fn.col("total_impressions")))

In [14]:
site_df = df.groupBy("site_id", "date").agg(
    fn.sum("total_impressions").alias("total_impressions"),
    fn.sum("unpaid_total_impressions").alias("unpaid_total_impressions"),
    fn.sum("total_revenue").alias("total_revenue"),
).withColumn(
    "fill_rate", (fn.col("total_impressions") - fn.col("unpaid_total_impressions")) / fn.col("total_impressions")
).withColumn(
    "CPM", fn.col("total_revenue") * 1000 / (fn.col("total_impressions") - fn.col("unpaid_total_impressions"))
).sort("site_id", "date")

In [15]:
site_df.show()

+-------+----------+-----------------+------------------------+------------------+------------------+------------------+
|site_id|      date|total_impressions|unpaid_total_impressions|     total_revenue|         fill_rate|               CPM|
+-------+----------+-----------------+------------------------+------------------+------------------+------------------+
|    342|2019-06-01|            17394|                     896|42.542799999999964|0.9484879843624239|2.5786640804946033|
|    342|2019-06-02|            29910|                    2028| 77.79620000000003| 0.932196589769308| 2.790194390646296|
|    342|2019-06-03|            32938|                    2348| 83.68319999999996|0.9287145546177668| 2.735639097744359|
|    342|2019-06-04|            17954|                    1242| 36.79819999999999|0.9308232148824774| 2.201902824317855|
|    342|2019-06-05|            15992|                     790|             36.45| 0.950600300150075|2.3977108275226944|
|    342|2019-06-06|            

In [11]:
advertiser_df = df.groupBy("advertiser_id").agg(
    fn.sum("total_impressions").alias("total_impressions"),
    fn.sum("unpaid_total_impressions").alias("unpaid_total_impressions"),
    fn.sum("total_revenue").alias("total_cost"),
).withColumn(
    "fill_rate", (fn.col("total_impressions") - fn.col("unpaid_total_impressions")) / fn.col("total_impressions")
).withColumn(
    "CPM", fn.col("total_cost") * 1000 / (fn.col("total_impressions") - fn.col("unpaid_total_impressions"))
)

In [12]:
advertiser_df.show()

+-------------+-----------------+------------------------+------------------+--------------------+------------------+
|advertiser_id|total_impressions|unpaid_total_impressions|        total_cost|           fill_rate|               CPM|
+-------------+-----------------+------------------------+------------------+--------------------+------------------+
|         2635|             9582|                    9582|               0.0|                 0.0|              null|
|         2637|           116676|                  116676|               0.0|                 0.0|              null|
|           16|          1014464|                     420| 4611.875399999995|  0.9995859882657245| 4.548003242462848|
|          139|            63340|                     428|209.32300000000006|  0.9932428165456267|3.3272348677517813|
|         2640|            39050|                   39050|               0.0|                 0.0|              null|
|           96|           299830|                      1